In [2]:
from IPython.utils import io
with io.capture_output() as output_03:
    %run '03_2d_spectra.ipynb'

In [11]:
to_dump = ['e_scale', 'fg_0_sub', 'fg_0_sub_err']
dump_dir = '08_ascii_files'

In [12]:
for var in to_dump:
    with open(dump_dir +'/' + var + '.txt','w') as f:
        for line in eval(var):
            f.write(str(line) + '\n')
            